In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # visualization
import seaborn as sns; sns.set() # graphing data

import warnings
warnings.filterwarnings("ignore")

In [16]:
import openpyxl

In [84]:
week_df = pd.read_csv('d:\homework\FIN3080\HW3\TRD_Week.csv')
week_df.head(4)

,Stkcd,Trdwnt,Wretnd,Markettype
0,1,2017-01,0.003297,4
1,1,2017-02,0.003286,4
2,1,2017-03,0.006550,4
3,1,2017-04,0.011931,4


In [ ]:
#转换日期格式
# week_df['Trdwnt'] = pd.to_datetime(week_df['Trdwnt'])

# 提取年月信息

# week_df['YW'] = week_df['Trdwnt'].dt.to_period('W')


In [85]:
def filter_rows(group):
    return group[(group['Markettype'] == 4) | (group['Markettype'] == 1) | (group['Markettype'] == 64)]

week_df_1 = week_df.groupby('Trdwnt').apply(filter_rows).reset_index(drop=True)


In [86]:
week_df_1.head(10)

,Stkcd,Trdwnt,Wretnd,Markettype
0,1,2017-01,0.003297,4
1,2,2017-01,0.004380,4
2,4,2017-01,-0.020935,4
3,5,2017-01,0.039941,4
4,6,2017-01,0.059259,4
5,7,2017-01,-0.007023,4
6,8,2017-01,0.017167,4
7,9,2017-01,0.014479,4
8,10,2017-01,0.002601,4
9,11,2017-01,0.081153,4


In [11]:
# 按 'Trdwnt' 列分组，计算 'Wretnd' 列的平均值
average_wretnd = week_df.groupby('Trdwnt')['Wretnd'].mean()


In [19]:
average_wretnd_df = average_wretnd.reset_index()

In [20]:
average_wretnd_df.head(10)
# # 将结果转换为字典形式，以便每个 'Trdwnt' 值对应一个平均值
# average_wretnd_dict = average_wretnd.to_dict()

# # 打印每个 'Trdwnt' 值对应的平均值
# for trdwnt, avg_wretnd in average_wretnd_dict.items():
#     print(f"Trdwnt: {trdwnt}, Average Wretnd: {avg_wretnd}")

,Trdwnt,Wretnd
0,2017-01,0.008763
1,2017-02,-0.043352
2,2017-03,-0.022521
3,2017-04,0.025113
4,2017-05,-0.004104
5,2017-06,0.036176
6,2017-07,0.000613
7,2017-08,0.046298
8,2017-09,0.011129
9,2017-10,0.018707


进入重点！

In [120]:
rf_df = pd.read_excel('d:\homework\FIN3080\HW3\weekly_risk_free_rate.xlsx')

In [121]:
rf_df.head(5)

,trading_date_yw,risk_free_return
0,2017-01-01,0.000571
1,2017-01-08,0.000738
2,2017-01-15,0.000543
3,2017-01-22,0.000774
4,2017-01-29,0.000802


In [110]:
# # 将 'trading_date_yw' 列转换为日期时间类型
# rf_df['trading_date_yw'] = pd.to_datetime(rf_df['trading_date_yw'])

# # 将 'trading_date_yw' 列转换为每周周期
# rf_df['trading_date_yw'] = rf_df['trading_date_yw'].dt.to_period('W')

# # 使用 groupby 方法按照 'trading_date_yw' 列进行分组，并计算每周的平均值
# weekly_rf_df = rf_df.groupby('trading_date_yw').mean()

In [111]:
# weekly_rf_df.head(5)

,risk_free_return
trading_date_yw,
2016-12-26/2017-01-01,0.000571
2017-01-02/2017-01-08,0.000738
2017-01-09/2017-01-15,0.000543
2017-01-16/2017-01-22,0.000774
2017-01-23/2017-01-29,0.000802


In [122]:
condition = ~(rf_df['trading_date_yw'].isin(["2017-01-01", "2018-01-01",  "2020-01-01",  "2022-01-01"]))

filtered_rf_df = rf_df[condition].reset_index(drop=True)

In [123]:
filtered_rf_df.head(5)

,trading_date_yw,risk_free_return
0,2017-01-08,0.000738
1,2017-01-15,0.000543
2,2017-01-22,0.000774
3,2017-01-29,0.000802
4,2017-02-05,0.000817


In [124]:
merged_df = pd.concat([average_wretnd_df, filtered_rf_df], axis=1)

In [125]:
pd.set_option('display.max_rows',300)
merged_df # 怎么最后几个还没对上啊 太诡异了吧 终于对上了。。。

,Trdwnt,Wretnd,trading_date_yw,risk_free_return
0,2017-01,0.008763,2017-01-08,0.000738
1,2017-02,-0.043352,2017-01-15,0.000543
2,2017-03,-0.022521,2017-01-22,0.000774
3,2017-04,0.025113,2017-01-29,0.000802
4,2017-05,-0.004104,2017-02-05,0.000817
...,...,...,...,...
303,2022-49,0.030856,2022-11-26,0.000444
304,2022-50,-0.007716,2022-12-03,0.000471
305,2022-51,-0.017675,2022-12-10,0.000477
306,2022-52,-0.052590,2022-12-17,0.000459


合并总表？

In [182]:
# 确保两个表有相同的股票代码列
common_w = pd.merge(week_df_1[['Trdwnt']].drop_duplicates(), merged_df[['Trdwnt']].drop_duplicates(), how='inner', on='Trdwnt')

# 根据相同的股票代码和年月进行合并
total_df = pd.merge(week_df_1[week_df_1['Trdwnt'].isin(common_w['Trdwnt'])], merged_df[merged_df['Trdwnt'].isin(common_w['Trdwnt'])], on=['Trdwnt'], how='left')


In [183]:
total_df

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return
0,1,2017-01,0.003297,4,0.008763,2017-01-08,0.000738
1,2,2017-01,0.004380,4,0.008763,2017-01-08,0.000738
2,4,2017-01,-0.020935,4,0.008763,2017-01-08,0.000738
3,5,2017-01,0.039941,4,0.008763,2017-01-08,0.000738
4,6,2017-01,0.059259,4,0.008763,2017-01-08,0.000738
...,...,...,...,...,...,...,...
727980,601995,2022-53,0.003949,1,0.008834,2022-12-24,0.000432
727981,601996,2022-53,0.018868,1,0.008834,2022-12-24,0.000432
727982,601997,2022-53,0.016667,1,0.008834,2022-12-24,0.000432
727983,601998,2022-53,0.020492,1,0.008834,2022-12-24,0.000432


In [184]:
# 将 'Trdwnt' 列转换为日期时间类型
total_df['Trdwnt'] = pd.to_datetime(total_df['Trdwnt'] + '-1', format='%Y-%W-%w')

# 根据 'Trdwnt' 列分成相对均匀的三组
total_df['Trdwnt_group'] = pd.qcut(total_df['Trdwnt'], q=3, labels=['Group1', 'Group2', 'Group3'])

In [185]:
total_df

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group
0,1,2017-01-02,0.003297,4,0.008763,2017-01-08,0.000738,Group1
1,2,2017-01-02,0.004380,4,0.008763,2017-01-08,0.000738,Group1
2,4,2017-01-02,-0.020935,4,0.008763,2017-01-08,0.000738,Group1
3,5,2017-01-02,0.039941,4,0.008763,2017-01-08,0.000738,Group1
4,6,2017-01-02,0.059259,4,0.008763,2017-01-08,0.000738,Group1
...,...,...,...,...,...,...,...,...
727980,601995,2023-01-02,0.003949,1,0.008834,2022-12-24,0.000432,Group3
727981,601996,2023-01-02,0.018868,1,0.008834,2022-12-24,0.000432,Group3
727982,601997,2023-01-02,0.016667,1,0.008834,2022-12-24,0.000432,Group3
727983,601998,2023-01-02,0.020492,1,0.008834,2022-12-24,0.000432,Group3


In [186]:
print(total_df.isnull().sum())

Stkcd                 0
Trdwnt                0
Wretnd_x            354
Markettype            0
Wretnd_y              0
trading_date_yw       0
risk_free_return      0
Trdwnt_group          0
dtype: int64


In [187]:
mean_value = total_df['Wretnd_x'].mean()
total_df['Wretnd_x'].fillna(mean_value, inplace=True)


In [188]:
import statsmodels.api as sm

# 按照Stkcd和Trdwnt_group分组，筛选出Group1的数据
group1_data = total_df[total_df['Trdwnt_group'] == 'Group1']

# 创建一个空的DataFrame来存储回归结果
beta_df = pd.DataFrame(columns=['Stkcd', 'Beta'])

# 对每个Stkcd的Group1数据进行回归分析，计算β值
for stkcd, group_data in group1_data.groupby('Stkcd'):
    X = group_data['Wretnd_y']
    y = group_data['Wretnd_x']
    
    X = sm.add_constant(X)  # 添加常数项
    model = sm.OLS(y, X).fit()  # 普通最小二乘法拟合
    beta = model.params['Wretnd_y']  # 获取斜率参数，即β值
    
    # 将结果添加到beta_df中
    beta_df = beta_df.append({'Stkcd': stkcd, 'Beta': beta}, ignore_index=True)

# 将得到的β值回填到DataFrame中对应的Stkcd行上
total_df = total_df.merge(beta_df, on='Stkcd', how='left')


In [139]:
# # 将 Group 2 和 Group 3 的 Beta 列都设置为 0 记笔记
# total_df.loc[total_df['Trdwnt_group'].isin(['Group2', 'Group3']), 'Beta'] = 0

In [189]:
total_df# 回填的时候

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta
0,1,2017-01-02,0.003297,4,0.008763,2017-01-08,0.000738,Group1,0.164861
1,2,2017-01-02,0.004380,4,0.008763,2017-01-08,0.000738,Group1,0.324017
2,4,2017-01-02,-0.020935,4,0.008763,2017-01-08,0.000738,Group1,0.572474
3,5,2017-01-02,0.039941,4,0.008763,2017-01-08,0.000738,Group1,0.773332
4,6,2017-01-02,0.059259,4,0.008763,2017-01-08,0.000738,Group1,0.791440
...,...,...,...,...,...,...,...,...,...
727980,601995,2023-01-02,0.003949,1,0.008834,2022-12-24,0.000432,Group3,NaN
727981,601996,2023-01-02,0.018868,1,0.008834,2022-12-24,0.000432,Group3,0.800798
727982,601997,2023-01-02,0.016667,1,0.008834,2022-12-24,0.000432,Group3,0.244807
727983,601998,2023-01-02,0.020492,1,0.008834,2022-12-24,0.000432,Group3,0.133049


In [216]:
# 将 Beta 列分成 10 组并添加分组标签
total_df['Beta_group'] = pd.qcut(total_df['Beta'], q=10, labels=False, duplicates='drop')



In [217]:
total_df.head(-20)

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf
0,1,2017-01-02,0.003297,4,0.008763,2017-01-08,0.000738,Group1,0.164861,0.0,0.008026
1,2,2017-01-02,0.004380,4,0.008763,2017-01-08,0.000738,Group1,0.324017,0.0,0.008026
2,4,2017-01-02,-0.020935,4,0.008763,2017-01-08,0.000738,Group1,0.572474,2.0,0.008026
3,5,2017-01-02,0.039941,4,0.008763,2017-01-08,0.000738,Group1,0.773332,5.0,0.008026
4,6,2017-01-02,0.059259,4,0.008763,2017-01-08,0.000738,Group1,0.791440,5.0,0.008026
...,...,...,...,...,...,...,...,...,...,...,...
727960,601928,2023-01-02,0.051793,1,0.008834,2022-12-24,0.000432,Group3,0.513709,1.0,0.008402
727961,601929,2023-01-02,0.010417,1,0.008834,2022-12-24,0.000432,Group3,0.680841,3.0,0.008402
727962,601933,2023-01-02,0.031073,1,0.008834,2022-12-24,0.000432,Group3,0.358484,0.0,0.008402
727963,601939,2023-01-02,0.018083,1,0.008834,2022-12-24,0.000432,Group3,0.070522,0.0,0.008402


In [222]:
# 对于Group2 对于每一周 计算每一组的平均值 收益 rpt  再新增一列 Wretnd_y-risk_free_return  再回归 用上面同样的方法

total_df['rm_rf'] = total_df['Wretnd_y'] - total_df["risk_free_return"]
# 筛选出 Trdwnt_group 为 Group2 的数据
group2_data = total_df[total_df['Trdwnt_group'] == 'Group2']



In [223]:
group2_data

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf
243376,1,2019-03-11,-0.036050,4,0.055655,2019-02-26,0.000503,Group2,0.164861,0.0,0.055152
243377,2,2019-03-11,-0.039944,4,0.055655,2019-02-26,0.000503,Group2,0.324017,0.0,0.055152
243378,4,2019-03-11,-0.033853,4,0.055655,2019-02-26,0.000503,Group2,0.572474,2.0,0.055152
243379,5,2019-03-11,0.079179,4,0.055655,2019-02-26,0.000503,Group2,0.773332,5.0,0.055152
243380,6,2019-03-11,-0.029412,4,0.055655,2019-02-26,0.000503,Group2,0.791440,5.0,0.055152
...,...,...,...,...,...,...,...,...,...,...,...
486265,601996,2021-02-08,0.058182,1,-0.056151,2021-02-05,0.000546,Group2,0.800798,5.0,-0.056697
486266,601997,2021-02-08,0.053708,1,-0.056151,2021-02-05,0.000546,Group2,0.244807,0.0,-0.056697
486267,601998,2021-02-08,0.030593,1,-0.056151,2021-02-05,0.000546,Group2,0.133049,0.0,-0.056697
486268,601999,2021-02-08,-0.045126,1,-0.056151,2021-02-05,0.000546,Group2,0.954526,7.0,-0.056697


In [224]:
# 按照 Trdwnt 和 Trdwnt_group 分组，计算每组每周的收益平均值
group2_weekly_mean = group2_data.groupby(['Trdwnt', 'Beta_group'])['Wretnd_x'].mean().reset_index()


In [225]:
group2_weekly_mean

,Trdwnt,Beta_group,Wretnd_x
0,2019-03-11,0.0,0.004743
1,2019-03-11,1.0,0.007210
2,2019-03-11,2.0,0.017640
3,2019-03-11,3.0,0.019227
4,2019-03-11,4.0,0.043880
...,...,...,...
995,2021-02-08,5.0,-0.033687
996,2021-02-08,6.0,-0.046981
997,2021-02-08,7.0,-0.049823
998,2021-02-08,8.0,-0.046673


In [226]:
# 合并计算得到的每组每周收益平均值到原始的 group2_data 中
group2_data = group2_data.merge(group2_weekly_mean, on=['Trdwnt', 'Beta_group'], how='left', suffixes=('', '_mean'))

In [227]:
group2_data # 因为Group2 所以只有部分日期

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf,Wretnd_x_mean
0,1,2019-03-11,-0.036050,4,0.055655,2019-02-26,0.000503,Group2,0.164861,0.0,0.055152,0.004743
1,2,2019-03-11,-0.039944,4,0.055655,2019-02-26,0.000503,Group2,0.324017,0.0,0.055152,0.004743
2,4,2019-03-11,-0.033853,4,0.055655,2019-02-26,0.000503,Group2,0.572474,2.0,0.055152,0.017640
3,5,2019-03-11,0.079179,4,0.055655,2019-02-26,0.000503,Group2,0.773332,5.0,0.055152,0.035300
4,6,2019-03-11,-0.029412,4,0.055655,2019-02-26,0.000503,Group2,0.791440,5.0,0.055152,0.035300
...,...,...,...,...,...,...,...,...,...,...,...,...
242889,601996,2021-02-08,0.058182,1,-0.056151,2021-02-05,0.000546,Group2,0.800798,5.0,-0.056697,-0.033687
242890,601997,2021-02-08,0.053708,1,-0.056151,2021-02-05,0.000546,Group2,0.244807,0.0,-0.056697,-0.009600
242891,601998,2021-02-08,0.030593,1,-0.056151,2021-02-05,0.000546,Group2,0.133049,0.0,-0.056697,-0.009600
242892,601999,2021-02-08,-0.045126,1,-0.056151,2021-02-05,0.000546,Group2,0.954526,7.0,-0.056697,-0.049823


In [228]:
group2_data['rp_rf'] = group2_data['Wretnd_x_mean']-group2_data['risk_free_return']

In [229]:
# 创建一个空的 DataFrame 来存储回归结果
beta_df_2 = pd.DataFrame(columns=['Beta_group', 'Alpha_p', 'Alpha_p_t_value', 'Beta_p', 'Beta_p_t_value', 'R_squared'])

# 对每个 betagroup 的 Group2 数据进行回归分析，计算 αp、αp 的 t 值、βp、βp 的 t 值和 R-squared
for beta_group, group_data in group2_data.groupby(['Beta_group']):
    X = group_data['rm_rf']
    y = group_data['rp_rf']
    
    X = sm.add_constant(X)  # 添加常数项
    model = sm.OLS(y, X).fit()  # 普通最小二乘法拟合
    
    # 获取回归结果
    alpha_p = model.params['const']  # αp
    alpha_p_t_value = model.tvalues['const']  # αp 的 t 值
    beta_p = model.params['rm_rf']  # βp
    beta_p_t_value = model.tvalues['rm_rf']  # βp 的 t 值
    r_squared = model.rsquared  # R-squared
    
    # 将结果添加到 beta_df_2 中
    beta_df_2 = beta_df_2.append({'Beta_group': beta_group, 'Alpha_p': alpha_p, 
                                  'Alpha_p_t_value': alpha_p_t_value, 'Beta_p': beta_p, 
                                  'Beta_p_t_value': beta_p_t_value, 'R_squared': r_squared}, ignore_index=True)

# # 将得到的结果合并到 total_df 中
# total_df = total_df.merge(beta_df_2, on=['Beta_group', 'Trdwnt'], how='left')

In [230]:
beta_df_2

,Beta_group,Alpha_p,Alpha_p_t_value,Beta_p,Beta_p_t_value,R_squared
0,0.0,-0.001336,-14.974993,0.577914,234.130010,0.696557
1,1.0,-0.000369,-4.274027,0.627544,262.873014,0.745220
2,2.0,-0.000780,-8.243975,0.689754,263.486187,0.746192
3,3.0,-0.000936,-10.155739,0.707483,277.476148,0.765285
4,4.0,-0.001680,-19.548569,0.765183,321.804467,0.813240
5,5.0,-0.002377,-27.026871,0.787402,323.642680,0.814508
6,6.0,-0.001865,-22.072021,0.808669,345.774227,0.835126
7,7.0,-0.001627,-18.501594,0.853503,350.665129,0.838254
8,8.0,-0.001768,-19.938150,0.858617,349.861251,0.835150
9,9.0,-0.002723,-31.748770,0.914344,385.288485,0.857381


In [231]:
group3_data = total_df[total_df['Trdwnt_group'] == 'Group3']

# 按照 Trdwnt 和 Trdwnt_group 分组，计算每组每周的收益平均值
group3_weekly_mean = group3_data.groupby(['Trdwnt', 'Beta_group'])['Wretnd_x'].mean().reset_index()
# 合并计算得到的每组每周收益平均值到原始的 group2_data 中
group3_data = group3_data.merge(group3_weekly_mean, on=['Trdwnt', 'Beta_group'],how='left',suffixes=('', '_mean'))

In [232]:
group3_data['rp_rf'] = group3_data['Wretnd_x_mean']-group3_data['risk_free_return']

In [233]:
group3_data

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf,Wretnd_x_mean,rp_rf
0,1,2021-02-15,-0.044525,4,0.021677,2021-02-12,0.000553,Group3,0.164861,0.0,0.021124,0.030503,0.029950
1,2,2021-02-15,-0.001731,4,0.021677,2021-02-12,0.000553,Group3,0.324017,0.0,0.021124,0.030503,0.029950
2,4,2021-02-15,0.009501,4,0.021677,2021-02-12,0.000553,Group3,0.572474,2.0,0.021124,0.034225,0.033672
3,5,2021-02-15,0.043956,4,0.021677,2021-02-12,0.000553,Group3,0.773332,5.0,0.021124,0.032736,0.032183
4,6,2021-02-15,0.010417,4,0.021677,2021-02-12,0.000553,Group3,0.791440,5.0,0.021124,0.032736,0.032183
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241710,601995,2023-01-02,0.003949,1,0.008834,2022-12-24,0.000432,Group3,NaN,NaN,0.008402,NaN,NaN
241711,601996,2023-01-02,0.018868,1,0.008834,2022-12-24,0.000432,Group3,0.800798,5.0,0.008402,0.011983,0.011550
241712,601997,2023-01-02,0.016667,1,0.008834,2022-12-24,0.000432,Group3,0.244807,0.0,0.008402,0.011327,0.010894
241713,601998,2023-01-02,0.020492,1,0.008834,2022-12-24,0.000432,Group3,0.133049,0.0,0.008402,0.011327,0.010894


In [234]:
rp_rf_mean = group3_data.groupby(['Beta_group'])['rp_rf'].mean().reset_index()

In [235]:
group3_data = group3_data.merge(rp_rf_mean, on=['Beta_group'], how='left', suffixes=('', '_mean'))

In [237]:
pd.set_option('display.max_rows',500)
group3_data.head(5)

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf,Wretnd_x_mean,rp_rf,rp_rf_mean
0,1,2021-02-15,-0.044525,4,0.021677,2021-02-12,0.000553,Group3,0.164861,0.0,0.021124,0.030503,0.029950,0.001921
1,2,2021-02-15,-0.001731,4,0.021677,2021-02-12,0.000553,Group3,0.324017,0.0,0.021124,0.030503,0.029950,0.001921
2,4,2021-02-15,0.009501,4,0.021677,2021-02-12,0.000553,Group3,0.572474,2.0,0.021124,0.034225,0.033672,0.001945
3,5,2021-02-15,0.043956,4,0.021677,2021-02-12,0.000553,Group3,0.773332,5.0,0.021124,0.032736,0.032183,0.003351
4,6,2021-02-15,0.010417,4,0.021677,2021-02-12,0.000553,Group3,0.791440,5.0,0.021124,0.032736,0.032183,0.003351


In [240]:
unique_df = group3_data.drop_duplicates(subset=['rp_rf_mean'])

In [242]:
unique_df.dropna(inplace=True)

In [243]:
unique_df

,Stkcd,Trdwnt,Wretnd_x,Markettype,Wretnd_y,trading_date_yw,risk_free_return,Trdwnt_group,Beta,Beta_group,rm_rf,Wretnd_x_mean,rp_rf,rp_rf_mean
0,1,2021-02-15,-0.044525,4,0.021677,2021-02-12,0.000553,Group3,0.164861,0.0,0.021124,0.030503,0.029950,0.001921
2,4,2021-02-15,0.009501,4,0.021677,2021-02-12,0.000553,Group3,0.572474,2.0,0.021124,0.034225,0.033672,0.001945
3,5,2021-02-15,0.043956,4,0.021677,2021-02-12,0.000553,Group3,0.773332,5.0,0.021124,0.032736,0.032183,0.003351
5,7,2021-02-15,-0.019455,4,0.021677,2021-02-12,0.000553,Group3,0.706738,4.0,0.021124,0.024581,0.024027,0.003252
7,9,2021-02-15,-0.001225,4,0.021677,2021-02-12,0.000553,Group3,0.993228,8.0,0.021124,0.030109,0.029556,0.003262
8,10,2021-02-15,-0.008086,4,0.021677,2021-02-12,0.000553,Group3,0.846638,6.0,0.021124,0.029249,0.028696,0.004324
11,14,2021-02-15,0.006623,4,0.021677,2021-02-12,0.000553,Group3,0.947007,7.0,0.021124,0.030687,0.030134,0.003327
15,20,2021-02-15,0.054020,4,0.021677,2021-02-12,0.000553,Group3,1.312586,9.0,0.021124,0.019470,0.018916,0.003490
19,26,2021-02-15,0.092308,4,0.021677,2021-02-12,0.000553,Group3,0.635775,3.0,0.021124,0.030528,0.029975,0.002751
21,28,2021-02-15,0.047890,4,0.021677,2021-02-12,0.000553,Group3,0.520194,1.0,0.021124,0.034530,0.033977,0.002036


In [244]:

# 假设你的两列分别是 column1 和 column2
column1_unique = unique_df['Beta_group']
column2_unique = unique_df['rp_rf_mean']

# 使用 pd.concat() 合并两列为一个新的 DataFrame
new3_df = pd.concat([column1_unique, column2_unique], axis=1)



In [245]:

# 假设你的两列分别是 column3 和 column4
column3_unique = beta_df_2['Beta_group']
column4_unique = beta_df_2['Beta_p']

# 使用 pd.concat() 合并两列为一个新的 DataFrame
new4_df = pd.concat([column3_unique, column4_unique], axis=1)



In [246]:
regression3_df = pd.merge(new3_df, new4_df, on='Beta_group')

In [248]:
regression3_df

,Beta_group,rp_rf_mean,Beta_p
0,0.0,0.001921,0.577914
1,2.0,0.001945,0.689754
2,5.0,0.003351,0.787402
3,4.0,0.003252,0.765183
4,8.0,0.003262,0.858617
5,6.0,0.004324,0.808669
6,7.0,0.003327,0.853503
7,9.0,0.003490,0.914344
8,3.0,0.002751,0.707483
9,1.0,0.002036,0.627544


In [254]:
# 创建一个空的 DataFrame 来存储回归结果
beta_df_3 = pd.DataFrame(columns=[ 'Gamma_0', 'Gamma_1','Gamma_0_t_value','Gamma_1_t_value', 'R_squared', 'F_statistic', 'P_value'])

# 对每个 beta_group 进行回归分析，y 是平均收益 rp_rf_mean，X 是 Beta_p
X = regression3_df['Beta_p']
y = regression3_df['rp_rf_mean']

X = sm.add_constant(X)  # 添加常数项
model = sm.OLS(y, X).fit()  # 普通最小二乘法拟合

# 获取回归结果
gamma_0 = model.params['const']  # γ0
gamma_1 = model.params['Beta_p']  # γ1
r_squared = model.rsquared  # R-squared
f_statistic = model.fvalue  # F 检验值
p_value = model.f_pvalue  # F 检验值对应的 P-value
gamma_0_t_value = model.tvalues['const']  # γ0 的 t 值
gamma_1_t_value = model.tvalues['Beta_p']  # γ1 的 t 值

# 将结果添加到 beta_df_2 中
beta_df_3 = beta_df_3.append({'Gamma_0': gamma_0, 
                              'Gamma_1': gamma_1,'Gamma_0_t_value': gamma_0_t_value, 'Gamma_1_t_value': gamma_1_t_value, 'R_squared': r_squared, 
                              'F_statistic': f_statistic, 'P_value': p_value}, ignore_index=True)


In [255]:
beta_df_3

,Gamma_0,Gamma_1,Gamma_0_t_value,Gamma_1_t_value,R_squared,F_statistic,P_value
0,-0.001556,0.005957,-1.33334,3.909797,0.656453,15.286512,0.004482
